In [1]:
import torch
from IPython.display import clear_output
import time

def knapsack_specialized(xi, v, w, C):
    """
    Solves a specialized knapsack problem using a specialized method in a vectorized way

    Args:
        xi (torch.Tensor): xi variables.
        v (torch.Tensor): Quantization vector.
        w (torch.Tensor): Weight vector.
        C (int): Number of buckets of quantization.

    Returns:
        tuple: Optimal allocation (x_opt), optimal multipliers (lambda_opt), and objective values.
    """
    
    b_list = []
    b = 0

    # Compute breakpoint vector x_plus
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0

        if b != C - 1:
            b_list.append(int(b))

        if b + 1 > C - 1:
            break
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1

    # Determine optimal allocation based on w
    w_idx = torch.searchsorted(v, w) 
    indices_breakpoints = torch.nonzero(x_plus == 1).squeeze()

    # Creation of masks for extreme cases
    mask_right = w > v[-1]
    mask_left = w < v[0]

    # Find indices using searchsorted
    search_idx = torch.searchsorted(indices_breakpoints, w_idx)

    # Ensure that the indices are valid
    search_idx = torch.clamp(search_idx, 1, len(indices_breakpoints) - 1)

    # Initialize idx_right and idx_left with the result of the search
    idx_right = indices_breakpoints[search_idx]
    idx_left = indices_breakpoints[search_idx - 1]

    # Correct the indices for extreme cases
    idx_right = torch.where(mask_right, indices_breakpoints[-1], idx_right)
    idx_left = torch.where(mask_right, indices_breakpoints[-1], idx_left)

    # Correct the indices for the case when w < v[0]
    idx_right = torch.where(mask_left, indices_breakpoints[0], idx_right)
    idx_left = torch.where(mask_left, indices_breakpoints[0], idx_left)

    # Compute convex combination for optimal solution
    x1, x2 = torch.zeros(2, len(w), C, dtype=torch.float32)

    x1[torch.arange(len(w)), idx_left] = 1
    x2[torch.arange(len(w)), idx_right] = 1

    numerator = w - torch.matmul(x2, v)
    denominator = torch.matmul((x1 - x2), v)
    theta = numerator / denominator

    mask_equal = (x1 == x2)
    theta_expanded = theta.unsqueeze(1)
    x_opt = torch.where(mask_equal, x1, x1 * theta_expanded + x2 * (1 - theta_expanded))

    # Compute optimal multipliers
    denominator = (v[idx_right] - v[idx_left])
    denominator_zero_mask = denominator == 0

    lambda_opt_nonzero = (xi[idx_right] - xi[idx_left]) / denominator
    lambda_opt_zero_full = xi / v
    lambda_opt_zero_full[0] = 0
    lambda_opt_zero = lambda_opt_zero_full[idx_left]

    lambda_opt = torch.where(denominator_zero_mask, lambda_opt_zero, lambda_opt_nonzero)

    # Compute objective function values
    objective_values = torch.matmul(x_opt, xi)

    return x_opt, lambda_opt, objective_values

# Function to compute phi and the gradient
def compute_phi_and_gradient(xi, v, w, C, upper_c):
    """
    Compute the function phi and its gradient.

    Parameters:
    xi: Current solution vector.
    v: Ordered vector of size C.
    w: Real value belonging to v.
    C: Problem dimension.
    upper_c: Upper bound for constraint handling.

    Returns:
    phi: Computed phi value.
    g: Gradient vector.
    """
    x_i_star, lambda_plus, phi_plus = knapsack_specialized(xi, v, w, C)
    sum_x_star = torch.sum(x_i_star, dim=0)
    c_star = torch.exp(torch.log(torch.tensor(2.0)) * xi - 1)
    c_star = torch.clamp(c_star, min=0.01, max=upper_c)
    
    # Compute phi terms
    phi1 = torch.sum(c_star * torch.log(c_star) / torch.log(torch.tensor(2.0)))
    phi2 = -torch.sum(xi * c_star)
    phi3 = torch.sum(xi * sum_x_star)
    phi = phi1 + phi2 + phi3
    
    # Compute the gradient (super-gradient)
    g = -(c_star - sum_x_star)
    return phi, g

In [2]:
# Initial parameters
C = 8
N = 44000
upper_c = N

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

xi_tmp = torch.sort(torch.rand(C, device=device))[0]  
xi = xi_tmp.clone()
v = torch.linspace(0, 1 - (1 / C), C, device=device)  
w = v[torch.randint(0, C, (N,), device=device)]  
#w = torch.rand(N, device=device)

#print("xi:", xi)
#print("v:", v)
#print("w:", w)

In [3]:
max_iterations = 1000
xi = xi_tmp.clone()

# Start
start_time = time.time()

# Main subgradient loop
#print("xi:", xi)
for iteration in range(1, max_iterations + 1):
    # Compute phi and gradient at y
    phi, g = compute_phi_and_gradient(xi, v, w, C, upper_c)

    # Proximal update step
    xi = xi + (1 / iteration) * g  # Proximal update
    
    # Display progress
    clear_output(wait=True)
    print(f"Iteration {iteration}: phi = {phi.item()}")
    #time.sleep(0.01)
    
training_time = time.time() - start_time
print(f'Time spent to find the optimum: {training_time:.2f} seconds\n')

Iteration 1000: phi = -174209.1875
Time spent to find the optimum: 3.30 seconds



In [4]:
max_iterations = 100
xi = xi_tmp.clone()

# Start
start_time = time.time()

# Main subgradient loop
#print("xi:", xi)
for iteration in range(1, max_iterations + 1):
    # Compute phi and gradient at y
    phi, g = compute_phi_and_gradient(xi, v, w, C, upper_c)

    # Proximal update step
    xi = xi + (1 / iteration) * g  # Proximal update
    
    # Display progress
    clear_output(wait=True)
    print(f"Iteration {iteration}: phi = {phi.item()}")
    #time.sleep(0.01)
    
training_time = time.time() - start_time
print(f'Time spent to find the optimum: {training_time:.2f} seconds\n')

Iteration 100: phi = -7917219.0
Time spent to find the optimum: 0.36 seconds



In [5]:
# Variables for FISTA
xi_prev = xi_tmp.clone()
xi = xi_tmp.clone()
t_prev = 1.0
zeta = 90000  # Regularization parameter

# Start
start_time = time.time()

# Main FISTA loop
for iteration in range(1, max_iterations + 1):
    # Accelerated step (momentum)
    t_current = (1 + torch.sqrt(torch.tensor(1) + 4 * t_prev**2)) / 2
    y = xi + ((t_prev - 1) / t_current) * (xi - xi_prev)
    
    # Compute phi and gradient at y
    phi, g = compute_phi_and_gradient(y, v, w, C, upper_c)
    
    # Proximal update step
    xi_next = y + (1 / zeta) * g  # Proximal update
    
    # Clipping to ensure constraints on xi
    xi_next = torch.clamp(xi_next, min=0.01, max=upper_c)
    
    # Display progress
    clear_output(wait=True)
    print(f"Iteration {iteration}: phi = {phi.item()}")
    #time.sleep(0.01)
    
    # Update for the next iteration
    xi_prev = xi.clone()
    xi = xi_next.clone()
    t_prev = t_current
    
    
training_time = time.time() - start_time
print(f'Time spent to find the optimum: {training_time:.2f} seconds\n')

Iteration 100: phi = 545534.0
Time spent to find the optimum: 0.35 seconds



In [6]:
import torch
from torch.linalg import norm
from IPython.display import clear_output
import time

# Initial parameters
zeta = 1e6  # Regularization parameter

xi = xi_tmp.clone()

# Parameters for the Bundle method
bundle_size = 5  # Maximum bundle size
bundle = []  # Initial bundle (list of points, phi values, and gradients)

# Start
start_time = time.time()

# Main loop of the Bundle Proximal method
for iteration in range(1, max_iterations + 1):
    # Compute phi and gradient at the current point
    phi, g = compute_phi_and_gradient(xi, v, w, C, upper_c)
    
    # Add the current point to the bundle
    bundle.append((xi.clone(), phi, g.clone()))
    if len(bundle) > bundle_size:
        bundle.pop(0)  # Remove the oldest point if the bundle exceeds the maximum size
    
    # Solve the subproblem (regularized quadratic minimization)
    bundle_points = torch.stack([item[0] for item in bundle])  # Bundle points
    bundle_phis = torch.tensor([item[1] for item in bundle])  # Phi values
    bundle_gradients = torch.stack([item[2] for item in bundle])  # Bundle gradients
    
    # Construct the regularized quadratic problem
    diff = xi - bundle_points
    model_phi = bundle_phis + torch.sum(bundle_gradients * diff, dim=1)
    proximal_term = (zeta / 2) * norm(diff, dim=1)**2
    subproblem_objective = model_phi + proximal_term
    
    # Find the new point by maximizing the model
    best_idx = torch.argmax(subproblem_objective)
    xi_next = bundle_points[best_idx] + (1 / zeta) * bundle_gradients[best_idx]
    
    # Clipping to ensure constraints on xi
    xi_next = torch.clamp(xi_next, min=0.01, max=upper_c)
    
    # Display progress
    clear_output(wait=True)
    print(f"Iteration {iteration}: phi = {phi.item()}")
    #time.sleep(0.01)
    
    # Update xi
    xi = xi_next.clone()
    
    
training_time = time.time() - start_time
print(f'Time spent to find the optimum: {training_time:.2f} seconds\n')

Iteration 100: phi = 20513.814453125
Time spent to find the optimum: 0.36 seconds

